# Project 3

## Part 2: Modeling

Model data for fun and profit.

### 0. Imports and Preliminaries

In [119]:
# imports
import pandas as pd
import numpy as np

# preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler

# models
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import MultinomialNB

# metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

# cross-validation
from sklearn.model_selection import train_test_split, cross_val_score

# pipelines, gridsearch
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# nltk - for stopwords and stemming
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import word_tokenize

# custom
import ipynb_utils as ipyutils

In [2]:
# load data
df = pd.read_json('../data/scrapes-clean.json', orient='index')

# convert time to datetime object
df['time'] = pd.to_datetime(df['time'], format=ipyutils.DATE_FMT)

In [3]:
# check that all looks good...
df.head()

,time,title,body-text,title-cc,title-wc,body-cc,body-wc,media,comments
0,2022-09-05,Newbie questions about ascendants and borders,"I'm new to actually learning astrology, not ju...",45,6,601,107,0,0
2,2022-09-05,Astrology and cognitive dissonance,Open to anyone who wouldn't mind sharing a rec...,34,4,323,49,0,1
3,2022-09-05,what do y’all think of persona charts?,"I feel a bit skeptical of them, since I feel l...",38,7,180,29,0,0
4,2022-09-05,RESOURCE REQUEST: Videos (or articles) with ti...,I think my problem is that I don’t know the pr...,160,24,597,94,0,2
5,2022-09-05,"people who have had saturn transit their 10th,...",How did it affect your career? Did it impact y...,64,12,116,22,0,11


In [4]:
# ... and that the right datatypes are showing
df.dtypes

time         datetime64[ns]
title                object
body-text            object
title-cc              int64
title-wc              int64
body-cc               int64
body-wc               int64
media                 int64
comments              int64
dtype: object

In [5]:
df.shape

(8413, 9)

### 0.5. Problem Statement

What characteristics of a post on Reddit are most predictive of the overall interaction on a thread (as measured by number of comments)?

Model will attempt to predict whether or not a given Reddit post will have above or below the median number of comments.

### 1. Generate Target

In [6]:
# median comments
median = np.median(df['comments'])
median

14.0

In [7]:
# target column
df['comments_gt_median'] = (df['comments'] > median).astype(int)
df['comments_gt_median'].value_counts()

0    4271
1    4142
Name: comments_gt_median, dtype: int64

In [8]:
df['comments_gt_median'].value_counts(normalize=True)

0    0.507667
1    0.492333
Name: comments_gt_median, dtype: float64

#### Baseline
Baseline is just about **50%**, as it should be since we are using median as split for determining high vs. low engagement.

In [9]:
# Store in obvious variable
TARGET = df['comments_gt_median']

### 1a. Split Time Column

Might want to check by month or day of week

In [10]:
df['day'] = df['time'].apply(ipyutils.get_day_of_week)

In [11]:
df['month'] = df['time'].apply(lambda x: x.month)

In [12]:
df[['day','month']].head()

,day,month
0,0,9
2,0,9
3,0,9
4,0,9
5,0,9


In [13]:
df['weekend'] = (df['day'] > 5).astype(int)
df['weekend'].head(10)

0     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
10    0
11    0
Name: weekend, dtype: int64

In [14]:
# Doing this just to be safe as I've gotten some weird row mismatches
# later on and not sure exactly why
df.reset_index(drop=True, inplace=True)

### 2. Train-Test Split

In [15]:
col_target = 'comments_gt_median'
cols_to_drop = ['time'] # don't need this any more
X = df.drop(columns=[col_target]+cols_to_drop)
y = df[col_target]

# split to train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    stratify=y,
                                                    test_size=0.2,
                                                    random_state=1)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6730, 11), (1683, 11), (6730,), (1683,))

### 3. Count Vectorize Text Fields

In [16]:
# utility functions for testing stemming - taken from course 
# materials 33-nlp-ii
def stem_tokenizer(doc):
    stemmer = PorterStemmer()
    tokens = word_tokenize(doc)
    return [stemmer.stem(t) for t in tokens]

def lemma_tokenizer(doc):
    lemmatizer = WordNetLemmatizer()
    tokens = word_tokenize(doc)
    return [lemmatizer.lemmatize(t) for t in tokens]

# I tried these on the CountVectorizer and they result in some bogus matches
# (like whitespace and punctuation). I don't have time to really look into this,
# and the scores from my tests on these were not very different from not using
# them, so I'm not going to use them this time around.

In [94]:
# get count vectorize tables
cv_params = {
    'token_pattern': ipyutils.PAT_TOKEN, # using standard CV token pattern
    'min_df': 15, # we don't want words that rarely appear
    'stop_words': stopwords.words('english'), # use nltk stopwords list
    'ngram_range': (1,3), # allow short phrases
    'tokenizer': None # tried stem_tokenizer, lemma_tokenizer - using neither
}
cv_title = CountVectorizer(**cv_params)
cv_body = CountVectorizer(**cv_params)
cv_alltext = CountVectorizer(**cv_params)

# title
train_title_cv = cv_title.fit_transform(X_train['title'])
test_title_cv = cv_title.transform(X_test['title'])

# body
train_body_cv = cv_body.fit_transform(X_train['body-text'])
test_body_cv = cv_body.transform(X_test['body-text'])

# title + body
train_alltext_cv = cv_alltext.fit_transform(X_train['title'] + ' ' + X_train['body-text'])
test_alltext_cv = cv_alltext.transform(X_test['title'] + ' ' + X_test['body-text'])

(train_title_cv.shape, test_title_cv.shape, 
train_body_cv.shape, test_body_cv.shape,
train_alltext_cv.shape, test_alltext_cv.shape)

((6730, 588),
 (1683, 588),
 (6730, 3994),
 (1683, 3994),
 (6730, 4355),
 (1683, 4355))

In [95]:
cv_title.get_feature_names_out()[-20:]

array(['week', 'well', 'western', 'whole', 'whole sign', 'within',
       'without', 'wondering', 'work', 'working', 'world', 'would',
       'wrong', 'year', 'years', 'yet', 'youtube', 'zodiac',
       'zodiac sign', 'zodiac signs'], dtype=object)

In [96]:
cv_alltext.get_feature_names_out()[-40:]

array(['wrong', 'wrote', 'www', 'www astro', 'www astro com',
       'www reddit', 'www reddit com', 'www youtube', 'www youtube com',
       'ya', 'yang', 'yeah', 'year', 'year ago', 'year half', 'years',
       'years ago', 'years old', 'yes', 'yesterday', 'yet', 'yin',
       'young', 'younger', 'youtu', 'youtube', 'youtube channel',
       'youtube com', 'youtube com watch', 'youtube video',
       'youtube videos', 'youtubers', 'zero', 'zeus', 'zodiac',
       'zodiac sign', 'zodiac signs', 'zodiacal', 'zodiacs', 'zone'],
      dtype=object)

### 3. Random Forest Classifier

In [97]:
title_rfc = RandomForestClassifier()
gs_params = {
    'n_estimators': [200, 300],
    'min_samples_leaf': [4, 5],
    'min_samples_split': [4, 5],
    'min_impurity_decrease': [0.0001, 0.001],
    'n_jobs': [-1],
    'random_state': [1]
}

# use gridsearch this time only to check best model params (takes a long time)
gs = GridSearchCV(title_rfc, gs_params, verbose=1, n_jobs=-1)

In [98]:
# model on titles only
gs.fit(train_title_cv, y_train)
print()
print(ipyutils.score_report(gs, 
                            (train_title_cv, y_train), 
                            (test_title_cv, y_test)))

Fitting 5 folds for each of 16 candidates, totalling 80 fits

Model Train Score (best): 0.7286775631500743
Model Test Score (best): 0.6428995840760546
Model Best Estimator: RandomForestClassifier(min_impurity_decrease=0.0001, min_samples_leaf=4,
                       min_samples_split=4, n_estimators=200, n_jobs=-1,
                       random_state=1)



In [99]:
# save best params to use for later models.
# done like this with variable alias because I previously hand-copied
# the params to a separate dictionary and needed the rfc_params variable for
# that dictionary
rfc_params = gs.best_params_

In [100]:
# model on body text only - use same best params from gridsearch
body_rfc = RandomForestClassifier(**rfc_params)
body_rfc.fit(train_body_cv, y_train)
print()
print(ipyutils.score_report(body_rfc, 
                            (train_body_cv, y_train), 
                            (test_body_cv, y_test)))


Model Train Score (best): 0.7885586924219911
Model Test Score (best): 0.6601307189542484



In [101]:
# model on all text
alltext_rfc = RandomForestClassifier(**rfc_params)
alltext_rfc.fit(train_alltext_cv, y_train)
print()
print(ipyutils.score_report(alltext_rfc, 
                            (train_alltext_cv, y_train), 
                            (test_alltext_cv, y_test)))


Model Train Score (best): 0.837592867756315
Model Test Score (best): 0.7011289364230541



#### Metrics

In [102]:
# title words
print(ipyutils.metrics_report(gs.best_estimator_, y_test, test_title_cv))

              precision    recall  f1-score   support

        high       0.66      0.62      0.64       854
         low       0.63      0.67      0.65       829

    accuracy                           0.64      1683
   macro avg       0.64      0.64      0.64      1683
weighted avg       0.64      0.64      0.64      1683

True Positives: 552
True Negatives: 530
False Positives: 324
False Negatives: 277



In [103]:
# body words
print(ipyutils.metrics_report(body_rfc, y_test, test_body_cv))

              precision    recall  f1-score   support

        high       0.70      0.58      0.63       854
         low       0.63      0.75      0.68       829

    accuracy                           0.66      1683
   macro avg       0.67      0.66      0.66      1683
weighted avg       0.67      0.66      0.66      1683

True Positives: 619
True Negatives: 492
False Positives: 362
False Negatives: 210



In [104]:
# all words
print(ipyutils.metrics_report(alltext_rfc, y_test, test_alltext_cv))

              precision    recall  f1-score   support

        high       0.72      0.68      0.70       854
         low       0.69      0.72      0.70       829

    accuracy                           0.70      1683
   macro avg       0.70      0.70      0.70      1683
weighted avg       0.70      0.70      0.70      1683

True Positives: 598
True Negatives: 582
False Positives: 272
False Negatives: 231



#### Analysis of Random Forest Classifier Score

Perhaps unsurprisingly, analyzing on the full text (body plus title) gave better prediction accuracy. However, for purposes of the problem statement, the title and body are possibly best kept separate, as reddit does not diplay the full body text by default, and searches only display titles. If we are looking for maximum engagement, we are more likely to reach the most number of users via the post titles rather than the post bodies.

Accuracy is better than baseline by 14-20 percentage points depending on the type of text field used. Since we are looking to increase engagement, the metrics concerning the quality of our positive outcomes are probably more important. Here, precision was higher for the high-engagement category, whereas recall was higher for the low-engagement category. In the alltext test, for example, the model correctly predicted high-engagement category 72% of the time, but only correctly predicted 68% of all high-engagement posts.

The model is overfit (which is probably to be expected from a decision-tree-based model).

#### Exploration of Model Results - Title Predictors

Or: what words were best predictors in the titles?

In [106]:
# title exploration - what words were best predictors in the titles?

# get predictions
title_preds_test = gs.best_estimator_.predict(test_title_cv)

# make dataframe from CountVectorizer sparse matrix
Xdf = pd.DataFrame(test_title_cv.A, 
                   columns=cv_title.get_feature_names_out(),
                   index=y_test.index)

# get metrics per word (see custom script ipynb_utils.py)
wc_df = ipyutils.wc_metrics(Xdf, y_test, title_preds_test, opts=[])

# filters - high word count, high accuracy, high recall
high_wc_filt = (wc_df['total'] > wc_df['total'].quantile(0.75))
high_accuracy_filt = (wc_df['accuracy'] >= 0.75)
high_recall_filt = (wc_df['recall'] >= 0.75)

# get results
wc_df[high_wc_filt & high_accuracy_filt & high_recall_filt].sort_values(by='accuracy', ascending=False)

,total,pct,correct,incorrect,diff,tp,tn,fp,fn,accuracy,recall,specificity,precision,f1
way,21,0.002697,20,1,19,11,9,0,1,0.952381,0.916667,1.000000,1.000000,0.956522
node,15,0.001927,14,1,13,8,6,1,0,0.933333,1.000000,0.857143,0.888889,0.941176
ascendant,18,0.002312,16,2,14,11,5,2,0,0.888889,1.000000,0.714286,0.846154,0.916667
placement,27,0.003468,24,3,21,22,2,2,1,0.888889,0.956522,0.500000,0.916667,0.936170
sagittarius,17,0.002183,15,2,13,13,2,2,0,0.882353,1.000000,0.500000,0.866667,0.928571
anyone else,19,0.002440,16,3,13,14,2,3,0,0.842105,1.000000,0.400000,0.823529,0.903226
really,19,0.002440,16,3,13,10,6,2,1,0.842105,0.909091,0.750000,0.833333,0.869565
born,20,0.002569,16,4,12,9,7,1,3,0.800000,0.750000,0.875000,0.900000,0.818182
chiron,15,0.001927,12,3,9,8,4,2,1,0.800000,0.888889,0.666667,0.800000,0.842105
sun moon,15,0.001927,12,3,9,9,3,3,0,0.800000,1.000000,0.500000,0.750000,0.857143


#### Analysis of Metrics

The above chart is compiled information from the words in the data set. Filters were created to select documents based on correct and incorrect predictions, among other metrics (see ipynb_utils.py script). The features (word counts) were then summed to get word counts per metric, and various other metrics were derived. This way these metrics could be explored on a word-by-word basis.

The above chart is sorted by accuracy score (high-to-low) and only shows words with total count in the 75th percentile.

### 4. Other Classifiers Comparison

I am comparing various other classifiers to see how general scores compare. Will select a second to use as a comparison classifier for future modeling.

#### ExtraTrees Classifier

In [111]:
title_etc = ExtraTreesClassifier()
# use same gs_params from random forest
title_etc_gs = GridSearchCV(title_etc, gs_params, verbose=1, n_jobs=-1)
title_etc_gs.fit(train_title_cv, y_train)
print(ipyutils.score_report(title_etc_gs,
                            (train_title_cv, y_train),
                            (test_title_cv, y_test)))

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Model Train Score (best): 0.7242199108469539
Model Test Score (best): 0.6399286987522281
Model Best Estimator: ExtraTreesClassifier(min_impurity_decrease=0.0001, min_samples_leaf=5,
                     min_samples_split=4, n_estimators=200, n_jobs=-1,
                     random_state=1)



In [112]:
etc_params = title_etc_gs.best_params_

In [113]:
body_etc = ExtraTreesClassifier(**etc_params)
body_etc.fit(train_body_cv, y_train)
print(ipyutils.score_report(body_etc,
                            (train_body_cv, y_train),
                            (test_body_cv, y_test)))

Model Train Score (best): 0.787518573551263
Model Test Score (best): 0.6708259061200238



In [114]:
alltext_etc = ExtraTreesClassifier(**etc_params)
alltext_etc.fit(train_alltext_cv, y_train)
print(ipyutils.score_report(alltext_etc,
                            (train_alltext_cv, y_train),
                            (test_alltext_cv, y_test)))

Model Train Score (best): 0.8346210995542348
Model Test Score (best): 0.6898395721925134



#### Analysis of Extra Trees Classifier Score

ExtraTrees performed slightly worse than Random Forest.

#### AdaBoost Classifier

In [115]:
# Ada Boost
ada = AdaBoostClassifier(random_state=1)
ada.fit(train_alltext_cv, y_train)
ada.score(train_alltext_cv, y_train), ada.score(test_alltext_cv, y_test)

(0.6705794947994056, 0.6102198455139631)

#### Gradient Boost Classifier

In [120]:
# Gradient Boost
gb = GradientBoostingClassifier()
gb.fit(train_alltext_cv, y_train)
gb.score(train_alltext_cv, y_train), gb.score(test_alltext_cv, y_test)

(0.7306092124814264, 0.6500297088532383)

#### K Nearest Neighbors Classifier

In [121]:
# K Neighbors
knc = KNeighborsClassifier(5)
knc.fit(train_alltext_cv, y_train)
knc.score(train_alltext_cv, y_train), knc.score(test_alltext_cv, y_test)

(0.7057949479940565, 0.5692216280451574)

#### Logistic Regression

In [122]:
# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(train_alltext_cv, y_train)
knc.score(train_alltext_cv, y_train), knc.score(test_alltext_cv, y_test)

(0.7057949479940565, 0.5692216280451574)

In [128]:
#### Multinomial Naive Bayes Classifier

In [123]:
# Multinomial Naive Bayes
mnb = MultinomialNB()
mnb.fit(train_alltext_cv, y_train)
mnb.score(train_alltext_cv, y_train), mnb.score(test_alltext_cv, y_test)

(0.7156017830609213, 0.6500297088532383)

#### Analysis of Other Classifiers on Word Vectors

Naive Bayes and Gradient Boost were tied on the test set. Other models were weaker performers. Due to less overfitting on Naive Bayes I will use that for future model comparisons.

### 4a. Other Features

There are a few other features I'd like to explore (word/character counts, for example).

Date/time features might not be appropriate here due to how Reddit works and the scraping process. Reddit no longer allows search by date, so I cannot get consecutive posts over time, and I am therefore trying to get as many posts I can via searches for words. Therefore, the post distribution over time that I get from my scrapes may not be the same as the actual post distribution over time, and there is no way to verify this with my current scraping process.

In [37]:
df.columns

Index(['time', 'title', 'body-text', 'title-cc', 'title-wc', 'body-cc',
       'body-wc', 'media', 'comments', 'comments_gt_median', 'day', 'month',
       'weekend'],
      dtype='object')

In [134]:
df.groupby('day').count() # decent spread, probably enough to be OK with here

,time,title,body-text,title-cc,title-wc,body-cc,body-wc,media,comments,comments_gt_median,month,weekend
day,,,,,,,,,,,,
0,698,698,698,698,698,698,698,698,698,698,698,698
1,584,584,584,584,584,584,584,584,584,584,584,584
2,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195,1195
3,1080,1080,1080,1080,1080,1080,1080,1080,1080,1080,1080,1080
4,986,986,986,986,986,986,986,986,986,986,986,986
5,2858,2858,2858,2858,2858,2858,2858,2858,2858,2858,2858,2858
6,1012,1012,1012,1012,1012,1012,1012,1012,1012,1012,1012,1012


In [135]:
df.groupby('month').count() # what's with september???

,time,title,body-text,title-cc,title-wc,body-cc,body-wc,media,comments,comments_gt_median,day,weekend
month,,,,,,,,,,,,
1,232,232,232,232,232,232,232,232,232,232,232,232
2,211,211,211,211,211,211,211,211,211,211,211,211
3,221,221,221,221,221,221,221,221,221,221,221,221
4,227,227,227,227,227,227,227,227,227,227,227,227
5,233,233,233,233,233,233,233,233,233,233,233,233
6,175,175,175,175,175,175,175,175,175,175,175,175
7,213,213,213,213,213,213,213,213,213,213,213,213
8,371,371,371,371,371,371,371,371,371,371,371,371
9,5637,5637,5637,5637,5637,5637,5637,5637,5637,5637,5637,5637


#### Word- and Character-counts and Media Indicator

I will take a brute-force approach and do a quick model on a lot of different feature sets and combinations, and select ones to take a closer look at based on model scores.

In [138]:
# I'm going to test all of the following feature combinations just to see if
# they show any major differences
col_opts = [
    ['media', 'title-cc', 'body-cc', 'title-wc', 'body-wc'],
    ['media'],
    ['title-cc', 'title-wc'],
    ['body-cc', 'body-wc'],
    ['title-cc', 'title-wc', 'body-cc', 'body-wc'],
    ['day'],
    ['month'],
    ['day', 'month'],
    ['title-cc', 'title-wc', 'body-cc', 'body-wc', 'day'],
    ['title-cc', 'title-wc', 'body-cc', 'body-wc', 'day', 'month']
]

In [139]:
# loop over each feature combination and run a model
for opt in col_opts:
    xrfc = RandomForestClassifier(**rfc_params)
    xrfc.fit(X_train[opt], y_train)
    print(f'{opt}\n\ttrain: {xrfc.score(X_train[opt], y_train)}'\
          + f'\n\ttest: {xrfc.score(X_test[opt], y_test)}')

['media', 'title-cc', 'body-cc', 'title-wc', 'body-wc']
	train: 0.8239227340267459
	test: 0.6226975638740344
['media']
	train: 0.5545319465081724
	test: 0.5472370766488414
['title-cc', 'title-wc']
	train: 0.6208023774145617
	test: 0.5561497326203209
['body-cc', 'body-wc']
	train: 0.6910846953937593
	test: 0.5430778371954843
['title-cc', 'title-wc', 'body-cc', 'body-wc']
	train: 0.8460624071322437
	test: 0.5870469399881164
['day']
	train: 0.5671619613670134
	test: 0.5680332739156269
['month']
	train: 0.5221396731054978
	test: 0.5086155674390969
['day', 'month']
	train: 0.6022288261515601
	test: 0.5941770647653001
['title-cc', 'title-wc', 'body-cc', 'body-wc', 'day']
	train: 0.8527488855869242
	test: 0.6030897207367796
['title-cc', 'title-wc', 'body-cc', 'body-wc', 'day', 'month']
	train: 0.8514115898959881
	test: 0.6125965537730244


In [140]:
# now let's do the same but adding the Vectorized title

# make vectorized title dataframes
train_title_cv_df = ipyutils.df_from_cv(cv_title, train_title_cv, X_train.index)
test_title_cv_df = ipyutils.df_from_cv(cv_title, test_title_cv, X_test.index)

# concat all column combos to vectorized title dataframes
combo_dfs = list()
for opt in col_opts:
    # 0 is train, 1 is test
    combo_dfs.append((ipyutils.easy_concat(X_train[opt], train_title_cv_df),
                      ipyutils.easy_concat(X_test[opt], test_title_cv_df)))

# check for integrity
for cdf in combo_dfs:
    print(cdf[0].shape, cdf[1].shape)

(6730, 593) (1683, 593)
(6730, 589) (1683, 589)
(6730, 590) (1683, 590)
(6730, 590) (1683, 590)
(6730, 592) (1683, 592)
(6730, 589) (1683, 589)
(6730, 589) (1683, 589)
(6730, 590) (1683, 590)
(6730, 593) (1683, 593)
(6730, 594) (1683, 594)


In [141]:
# run a model on each combo
for ix, cdf in enumerate(combo_dfs):
    opt = col_opts[ix]
    xrfc.fit(cdf[0], y_train)
    print(f'{opt}\n\ttrain:{xrfc.score(cdf[0], y_train)}\n'\
          + f'\ttest:{xrfc.score(cdf[1], y_test)}')

['media', 'title-cc', 'body-cc', 'title-wc', 'body-wc']
	train:0.7579494799405646
	test:0.6743909685086156
['media']
	train:0.7392273402674592
	test:0.6512180629827689
['title-cc', 'title-wc']
	train:0.736552748885587
	test:0.6339869281045751
['body-cc', 'body-wc']
	train:0.7401188707280832
	test:0.6530005941770648
['title-cc', 'title-wc', 'body-cc', 'body-wc']
	train:0.7527488855869242
	test:0.6589423648247178
['day']
	train:0.7378900445765231
	test:0.6446821152703506
['month']
	train:0.7323922734026745
	test:0.6428995840760546
['day', 'month']
	train:0.7445765230312036
	test:0.6642899584076055
['title-cc', 'title-wc', 'body-cc', 'body-wc', 'day']
	train:0.7557206537890044
	test:0.6654783125371361
['title-cc', 'title-wc', 'body-cc', 'body-wc', 'day', 'month']
	train:0.7610698365527488
	test:0.6714200831847891


In [150]:
# media + word/char counts seem to have most impact. 
# What are the stats?

# re-get score on [media + wc/cc + title_cv]
media_train = combo_dfs[0][0]
media_test = combo_dfs[0][1]

media_rfc = RandomForestClassifier(**rfc_params)
media_rfc.fit(media_train, y_train)
media_rfc.score(media_train, y_train), media_rfc.score(media_test, y_test)

(0.7579494799405646, 0.6743909685086156)

In [151]:
# get report on predictions for [media + wc/cc + title_cv]
media_preds_test = media_rfc.predict(media_test)
print(classification_report(y_test, media_preds_test))

              precision    recall  f1-score   support

           0       0.67      0.70      0.69       854
           1       0.68      0.65      0.66       829

    accuracy                           0.67      1683
   macro avg       0.67      0.67      0.67      1683
weighted avg       0.67      0.67      0.67      1683



In [157]:
# get report on [media + wc/cc] only
media_rfc = RandomForestClassifier(**rfc_params)
media_rfc.fit(X_train[col_opts[0]], y_train)
media_preds_test = media_rfc.predict(X_train[col_opts[0]])
print(classification_report(y_test, media_preds_test))

ValueError: Found input variables with inconsistent numbers of samples: [1683, 6730]

In [45]:
# Where media exists, there is no body text
df.loc[(df['media']==1) & (df['body-text'] != ''), ['media','body-text']].shape

(0, 2)

#### Analysis

From what I've seen so far, adding vectorized text data seems to even out the model a bit, with less overfitting than when using just word- and character-count features.

Here I was only using vectorized words from title fields. It is interesting that of all the additional features, the *media* feature (which states whether or not media such as images and video are in the post) is the most impactful in the absence of all others.

This begs the question, would a better predictor be a predictor on body text with media added, since it seems like media is an alternate body text type?

#### Exploring Media and Body Text

In [46]:
# Vectorized body sets: train_body_cv, test_body_cv
# Vectorized title sets: train_title_cv, test_title_cv
# Vectorized alltext sets: train_alltext_cv, test_alltext_cv
# Train media set: X_train['media']

train_body_cvdf = ipyutils.df_from_cv(cv_body, train_body_cv, y_train.index)
test_body_cvdf = ipyutils.df_from_cv(cv_body, test_body_cv, y_test.index)

train_media_df = X_train[['media']]
test_media_df = X_test[['media']]

# get titles for future exploration
train_title_cvdf = ipyutils.df_from_cv(cv_title, train_title_cv, y_train.index)
test_title_cvdf = ipyutils.df_from_cv(cv_title, test_title_cv, y_test.index)

In [47]:
# Combine body text and media tables
train_bodymedia_df = ipyutils.easy_concat(train_media_df, train_body_cvdf)
test_bodymedia_df = ipyutils.easy_concat(test_media_df, test_body_cvdf)

bodymedia_rfc = RandomForestClassifier(**rfc_params)
bodymedia_rfc.fit(train_bodymedia_df, y_train)
bodymedia_train_preds = bodymedia_rfc.predict(train_bodymedia_df)
bodymedia_test_preds = bodymedia_rfc.predict(test_bodymedia_df)
print("TRAIN\n", classification_report(y_train, bodymedia_train_preds), 
      "TEST\n", classification_report(y_test, bodymedia_test_preds))

TRAIN
               precision    recall  f1-score   support

           0       0.76      0.88      0.81      3417
           1       0.85      0.71      0.77      3313

    accuracy                           0.79      6730
   macro avg       0.80      0.79      0.79      6730
weighted avg       0.80      0.79      0.79      6730
 TEST
               precision    recall  f1-score   support

           0       0.65      0.73      0.69       854
           1       0.68      0.60      0.64       829

    accuracy                           0.67      1683
   macro avg       0.67      0.67      0.67      1683
weighted avg       0.67      0.67      0.67      1683



In [48]:
# can we do better with a different model?
bodymedia_nb = MultinomialNB()
bodymedia_nb.fit(train_bodymedia_df, y_train)
bodymedia_train_preds = bodymedia_nb.predict(train_bodymedia_df)
bodymedia_test_preds = bodymedia_nb.predict(test_bodymedia_df)
print("TRAIN\n", classification_report(y_train, bodymedia_train_preds),
      "TEST\n", classification_report(y_test, bodymedia_test_preds))

TRAIN
               precision    recall  f1-score   support

           0       0.72      0.79      0.75      3417
           1       0.76      0.68      0.72      3313

    accuracy                           0.74      6730
   macro avg       0.74      0.74      0.74      6730
weighted avg       0.74      0.74      0.74      6730
 TEST
               precision    recall  f1-score   support

           0       0.65      0.70      0.67       854
           1       0.66      0.62      0.64       829

    accuracy                           0.66      1683
   macro avg       0.66      0.66      0.66      1683
weighted avg       0.66      0.66      0.66      1683



In [49]:
# or a different model?
bodymedia_boost = AdaBoostClassifier()
bodymedia_boost.fit(train_bodymedia_df, y_train)
bodymedia_train_preds = bodymedia_boost.predict(train_bodymedia_df)
bodymedia_test_preds = bodymedia_boost.predict(test_bodymedia_df)
print("TRAIN\n", classification_report(y_train, bodymedia_train_preds),
      "TEST\n", classification_report(y_test, bodymedia_test_preds))

TRAIN
               precision    recall  f1-score   support

           0       0.65      0.78      0.71      3417
           1       0.71      0.57      0.63      3313

    accuracy                           0.68      6730
   macro avg       0.68      0.67      0.67      6730
weighted avg       0.68      0.68      0.67      6730
 TEST
               precision    recall  f1-score   support

           0       0.61      0.73      0.66       854
           1       0.65      0.52      0.58       829

    accuracy                           0.62      1683
   macro avg       0.63      0.62      0.62      1683
weighted avg       0.63      0.62      0.62      1683



In [50]:
# Finally, do one more test taking into account title words, body words, and media

# First prefix title and body words with title and body, respectively
train_title_cvdf = train_title_cvdf.add_prefix('(title) ')
train_body_cvdf = train_body_cvdf.add_prefix('(body) ')
test_title_cvdf = test_title_cvdf.add_prefix('(title) ')
test_body_cvdf = test_body_cvdf.add_prefix('(body) ')
train_title_cvdf.columns[:10]

Index(['(title) 10', '(title) 10th', '(title) 10th house', '(title) 11',
       '(title) 11th', '(title) 11th house', '(title) 12', '(title) 12h',
       '(title) 12th', '(title) 12th house'],
      dtype='object')

In [51]:
# make combination tables of media indicator, title words, and body words
train_megadf = pd.concat([train_media_df, train_title_cvdf, train_body_cvdf], axis=1)
test_megadf = pd.concat([test_media_df, test_title_cvdf, test_body_cvdf], axis=1)
train_megadf.shape, test_megadf.shape, y_train.shape, y_test.shape

((6730, 7052), (1683, 7052), (6730,), (1683,))

In [52]:
# run a Random Forest model
mega_rfc = RandomForestClassifier(**rfc_params)
mega_rfc.fit(train_megadf, y_train)
mega_train_preds = mega_rfc.predict(train_megadf)
mega_test_preds = mega_rfc.predict(test_megadf)
print("TRAIN\n", classification_report(y_train, mega_train_preds),
      "TEST\n", classification_report(y_test, mega_test_preds))

TRAIN
               precision    recall  f1-score   support

           0       0.81      0.86      0.83      3417
           1       0.85      0.79      0.82      3313

    accuracy                           0.83      6730
   macro avg       0.83      0.83      0.83      6730
weighted avg       0.83      0.83      0.83      6730
 TEST
               precision    recall  f1-score   support

           0       0.71      0.72      0.72       854
           1       0.71      0.70      0.70       829

    accuracy                           0.71      1683
   macro avg       0.71      0.71      0.71      1683
weighted avg       0.71      0.71      0.71      1683



In [53]:
# run a Naive Bayes model
mega_nb = MultinomialNB()
mega_nb.fit(train_megadf, y_train)
mega_train_preds = mega_nb.predict(train_megadf)
mega_test_preds = mega_nb.predict(test_megadf)
print("TRAIN\n", classification_report(y_train, mega_train_preds),
      "TEST\n", classification_report(y_test, mega_test_preds))

TRAIN
               precision    recall  f1-score   support

           0       0.75      0.79      0.77      3417
           1       0.77      0.73      0.75      3313

    accuracy                           0.76      6730
   macro avg       0.76      0.76      0.76      6730
weighted avg       0.76      0.76      0.76      6730
 TEST
               precision    recall  f1-score   support

           0       0.67      0.69      0.68       854
           1       0.67      0.65      0.66       829

    accuracy                           0.67      1683
   macro avg       0.67      0.67      0.67      1683
weighted avg       0.67      0.67      0.67      1683



In [54]:
# run Ada Boost
mega_ada = AdaBoostClassifier(n_estimators=100)
mega_ada.fit(train_megadf, y_train)
mega_train_preds = mega_ada.predict(train_megadf)
mega_test_preds = mega_ada.predict(test_megadf)
print("TRAIN\n", classification_report(y_train, mega_train_preds),
      "TEST\n", classification_report(y_test, mega_test_preds))

TRAIN
               precision    recall  f1-score   support

           0       0.69      0.78      0.73      3417
           1       0.74      0.64      0.68      3313

    accuracy                           0.71      6730
   macro avg       0.71      0.71      0.71      6730
weighted avg       0.71      0.71      0.71      6730
 TEST
               precision    recall  f1-score   support

           0       0.64      0.70      0.67       854
           1       0.66      0.59      0.62       829

    accuracy                           0.65      1683
   macro avg       0.65      0.65      0.65      1683
weighted avg       0.65      0.65      0.65      1683



#### Conclusions

The combining of body text, title text, and media indicator got me the highest score so far of over 70% accuracy on the test set. These results were from the Random Forest classifier, which has consistently outperformed all others on this data.

In [55]:
mega_rfc.fit(train_megadf, y_train)
mega_train_preds = mega_rfc.predict(train_megadf)
mega_test_preds = mega_rfc.predict(test_megadf)

In [58]:
# make my metrics dataframe
metrics_df = ipyutils.wc_metrics(test_megadf, y_test, mega_test_preds)

# set up filters
high_wc_filt = (metrics_df['total'] > 50)
high_accuracy_filt = (metrics_df['accuracy'] >= 0.75)
high_recall_filt = (metrics_df['recall'] >= 0.75)

# view results
metrics_df[high_wc_filt & high_accuracy_filt & high_recall_filt].sort_values(by='accuracy', ascending=False).head(20)

,total,pct,correct,incorrect,diff,tp,tn,fp,fn,accuracy,recall,specificity,precision,f1
(body) figure,79,0.000700,73,6,67,61,12,2,4,0.924051,0.938462,0.857143,0.968254,0.953125
(body) vs,58,0.000514,53,5,48,13,40,3,2,0.913793,0.866667,0.930233,0.812500,0.838710
(body) native,73,0.000647,66,7,59,64,2,6,1,0.904110,0.984615,0.250000,0.914286,0.948148
(body) soul,69,0.000612,60,9,51,24,36,2,7,0.869565,0.774194,0.947368,0.923077,0.842105
(body) lol,52,0.000461,45,7,38,31,14,4,3,0.865385,0.911765,0.777778,0.885714,0.898551
(body) father,75,0.000665,64,11,53,58,6,10,1,0.853333,0.983051,0.375000,0.852941,0.913386
(body) ideas,52,0.000461,44,8,36,26,18,5,3,0.846154,0.896552,0.782609,0.838710,0.866667
(body) 8th house,73,0.000647,61,12,49,51,10,9,3,0.835616,0.944444,0.526316,0.850000,0.894737
(body) emotions,60,0.000532,50,10,40,34,16,10,0,0.833333,1.000000,0.615385,0.772727,0.871795
(body) later,57,0.000505,47,10,37,17,30,7,3,0.824561,0.850000,0.810811,0.708333,0.772727


#### Other Stuff

In [59]:
df.groupby('day')['comments'].describe()

,count,mean,std,min,25%,50%,75%,max
day,,,,,,,,
0,698.0,37.614613,44.911348,0.0,10.0,22.0,48.00,442.0
1,584.0,28.755137,45.766167,0.0,6.0,14.0,31.00,561.0
2,1195.0,30.141423,47.413106,0.0,6.5,14.0,35.00,507.0
3,1080.0,57.019444,138.504473,0.0,9.0,25.0,64.25,3100.0
4,986.0,28.988844,53.898804,0.0,6.0,13.0,31.00,1075.0
5,2858.0,24.478307,71.773233,0.0,5.0,11.0,25.00,3138.0
6,1012.0,31.366601,44.111640,0.0,6.0,15.0,39.00,596.0


**NOTES** Thursday seems to be a hot day for comments, with a much higher mean, median, and maximum.

In [60]:
df.groupby('month')['comments'].describe()

,count,mean,std,min,25%,50%,75%,max
month,,,,,,,,
1,232.0,35.155172,46.704673,0.0,7.0,17.5,41.0,302.0
2,211.0,35.037915,55.930386,0.0,7.0,14.0,35.5,442.0
3,221.0,28.384615,40.605771,0.0,7.0,14.0,33.0,342.0
4,227.0,23.092511,29.126749,0.0,5.0,12.0,30.0,196.0
5,233.0,28.703863,33.165115,0.0,7.0,15.0,37.0,159.0
6,175.0,30.394286,41.814849,0.0,7.0,15.0,34.5,332.0
7,213.0,36.413146,55.160899,1.0,7.0,16.0,44.0,368.0
8,371.0,28.417790,43.926666,0.0,5.0,12.0,32.0,387.0
9,5637.0,33.625155,85.486574,0.0,6.0,14.0,35.0,3138.0


In [61]:
df[df['comments'] > 3000]

,time,title,body-text,title-cc,title-wc,body-cc,body-wc,media,comments,comments_gt_median,day,month,weekend
1811,2020-09-10,FIND YOUR BIG THREE TWIN (ORGANISED EDITION),READ:INSTRUCTIONS:I HAVE WRITTEN EVERY SUN AND...,44,7,1019,175,0,3100,1,3,9,0
4797,2021-09-11,FIND YOUR BIG THREE TWIN (ORGANISED EDITION),READ:\nINSTRUCTIONS:\n\nI HAVE WRITTEN EVERY S...,44,7,1030,175,0,3138,1,5,9,0


**NOTES** Months pretty stable, except there is one post in september 2021 that has over 3000 comments.